In [1]:
import os
import tensorflow as tf
import dagshub




In [2]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/vivek/work/GIT/end-to-end-mlflow'

In [8]:
os.environ["MLFLOW_REPO_OWNER"] = 'Vivek084c' 
os.environ["MLFLOW_REPO_NAME"] = 'end-to-end-mlflow'
os.environ["MLFLOW_MLFLOW"] = 'True'

In [9]:
# MLFLOW_REPO_OWNER = 'Vivek084c' 
# MLFLOW_REPO_NAME = 'end-to-end-mlflow'
# MLFLOW_MLFLOW = True

In [10]:


# import mlflow
# with mlflow.start_run():
#   mlflow.log_param('parameter name', 'value')
#   mlflow.log_metric('metric name', 1)

In [11]:
#loading the model
model = tf.keras.models.load_model("artifacts/training/model.h5")


2024-09-25 20:46:48.716193: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2024-09-25 20:46:48.717615: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-09-25 20:46:48.717666: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-09-25 20:46:48.717711: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-25 20:46:48.717846: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [12]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluateConfig:
    path_of_model : Path
    training_data : Path
    all_params : dict
    mlflow_uri : str
    params_image_size : list
    params_batch_size : int
    

In [13]:
from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml, create_directories, save_json


In [14]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_evaluation_config(self) -> EvaluateConfig:
        eval_config = EvaluateConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/data",
            mlflow_uri="",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [15]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse

In [16]:
class Evaluation:
    def __init__(self, config: EvaluateConfig):
        self.config = config

    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

    def vivek_log_into_mlflow(self):
        dagshub.init(repo_owner='Vivek084c', repo_name='end-to-end-mlflow', mlflow=True)
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )

            tracking_url_type_store = "asdf"
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [17]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.vivek_log_into_mlflow()
except Exception as e:
    raise e

Found 144 images belonging to 2 classes.


2024-09-25 20:46:49.796049: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.
/opt/miniconda3/envs/mlops/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 186ms/step - accuracy: 0.8971 - loss: 32.5736


Accessing as Vivek084c

Initialized MLflow to track repo "Vivek084c/end-to-end-mlflow"

Repository Vivek084c/end-to-end-mlflow initialized!

2024/09/25 20:47:15 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
/opt/miniconda3/envs/mlops/lib/python3.10/site-packages/_distutils_hack/__init__.py:32: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(
